In [10]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_pinecone import PineconeVectorStore
import os
os.environ["PINECONE_API_KEY"]="f0fe7f8f-d114-45e3-87cf-5c59f31dbb3e"

In [11]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="f0fe7f8f-d114-45e3-87cf-5c59f31dbb3e")
pc.create_index(
    name="test-example",
    dimension=4096, # Replace with your model dimensions
    metric="euclidean", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [12]:
embeddings = OllamaEmbeddings()
vectorstore = PineconeVectorStore(index_name="test-example", embedding=embeddings)

vectorstore.add_texts(["i worked at kensho"], namespace="harrison")
vectorstore.add_texts(["i worked at facebook"], namespace="ankush")

['3a6f3d7a-5991-46a6-8bd5-f572b79a166d']

In [13]:
# This will only get documents for Ankush
vectorstore.as_retriever(search_kwargs={"namespace": "ankush"}).get_relevant_documents(
    "where did i work?"
)

[Document(page_content='i worked at facebook')]

In [14]:
# This will only get documents for Harrison
vectorstore.as_retriever(
    search_kwargs={"namespace": "harrison"}
).get_relevant_documents("where did i work?")

[Document(page_content='i worked at kensho')]

In [15]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import (
    ConfigurableField,
    RunnableBinding,
    RunnableLambda,
    RunnablePassthrough,
)
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [16]:
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(base_url="http://localhost:11434/v1",model="llama2")

retriever = vectorstore.as_retriever()

In [17]:
configurable_retriever = retriever.configurable_fields(
    search_kwargs=ConfigurableField(
        id="search_kwargs",
        name="Search Kwargs",
        description="The search kwargs to use",
    )
)

In [18]:
chain = (
    {"context": configurable_retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [19]:
chain.invoke(
    "where did the user work?",
    config={"configurable": {"search_kwargs": {"namespace": "harrison"}}},
)

'\nBased on the provided context, the answer to the question "Where did the user work?" is "Kensho".'

In [20]:
chain.invoke(
    "where did the user work?",
    config={"configurable": {"search_kwargs": {"namespace": "ankush"}}},
)

'\nBased on the context provided, the answer to the question is "Facebook".'